### Step 0: Import Modules

In [790]:
import pandas as pd
from rectpack import newPacker
import random
import math

### Step 1: Define the Parameters

In [791]:
# number of tables
N = 3

# Minimum physical distane required between tables
d = 2

# capacity for each table
c = [2, 2, 2]

# width for each table
w = [1, 1, 2]

# account for physical distancing margin -> add 2m to the width
for i in range(len(w)):
    w[i] += d

# height for each table
h = [1, 2, 2]

# account for physical distancing margin -> add 2m to the height
for i in range(len(h)):
    h[i] += d

# Width of dining space
Rw = 10

# Height of dining space
Rh = 10

#Parameters of used tables
tables = []

#initialize neighbouring solution
neighbouringSolution = []


establishment = [(Rw, Rh)]

### Step 2: Initialize simulated annealing variables

In [792]:
#number of iterations that each temperature is held constant
m = 2

#initialize temperature
T = 100

#minimum amount of temperature level changes
k = 4

#temperature decrease constant
alpha = 0.2




### Step 3: Use construction heuristic to obtain initial solution

In [793]:
#STUBBED
#INITIALIZE CURRENT INCUMBENT
d = {'ifUsed' : [1,1,0,1,], 'x-coord' : [0,4,0,0], 'y-coord' : [0,0,0, 3], 'capacity' : [2,2,100,2], 'table space width' : [3,3,1,4], 'table space height' : [3,4,1,4]}
df = pd.DataFrame(data=d)
df







,ifUsed,x-coord,y-coord,capacity,table space width,table space height
0,1,0,0,2,3,3
1,1,4,0,2,3,4
2,0,0,0,100,1,1
3,1,0,3,2,4,4


### Step 4: Set up helper functions for simulated annealing algorithm

In [794]:
def setIncumbent():
    tableConfiguration = []
    for index, row in df.iterrows():
        tableConfiguration.append(row['ifUsed'])
    return tableConfiguration
    
#shuffles the configuration of used and unused tables randomly. Returns a shuffled ifUsed configuration
def shuffle(df):
    configuration = setIncumbent()
    random.shuffle(configuration)
    #print('neighbouring solution:', configuration)
    return configuration

def isFeasible(arrangement):
    tempTables = []
    
    for index, row in df.iterrows():
        if(arrangement[index]==1):
            tempTables.append((row['table space width'], row['table space height'], index))
    #print('neighbouring solution to be tested for feasibility: ', tempTables)
    
    #pack tables in the neighbouring solution
    packer = newPacker()
    for r in tempTables:
        packer.add_rect(*r)
        
    for b in establishment:
        packer.add_bin(*b)

    packer.pack()
    
    #if a table is missing after packing, it is not feasible
    if(len(packer.rect_list()) < len(tempTables)):
        #print('infeasible solution \n')
        return False
    else:
        #print('feasible solution \n')
        return True

#Generate feasible neighbouring solution by packing a subset of all available tables
def generateFeasibleSolution():
    solutionFound = False
    feasibleSolution = []
    while (solutionFound == False):
        shuffledConfiguration = shuffle(df)
        feasibleSolution = shuffledConfiguration
        solutionFound = isFeasible(shuffledConfiguration)
    
    if(solutionFound):
        return feasibleSolution
        

#Check how a feasible solution impacts the objective
def checkObjective(configuration):
    objective = 0
    index = 0
    for table in configuration:
        if configuration[index]==1:
            objective += df.at[index,'capacity']
        index += 1
    return (objective)

#Intitialize temperature schedule
def initializeTemperatureSchedule(T, alpha, k):
    latestTemp = T
    temperatureSchedule = [T]
    for i in range(k-1):
        temperatureSchedule.append(latestTemp*alpha)
        latestTemp = latestTemp*alpha
    return temperatureSchedule
    
    

### Step 4: Execute simulated annealing algorithm

In [795]:
def acceptCandidate(currentSolutionObjective, candidateSolutionObjective, T):
    #generate random number between 0 and 1
    randomNumber = random.uniform(0, 1)
    acceptanceProbability = math.exp((candidateSolutionObjective - currentSolutionObjective)/T)
    
    #if candidate solution is better, then accept it
    if (candidateSolutionObjective >= currentSolutionObjective):
        return 'betterCandidateAccepted'
    elif (randomNumber < acceptanceProbability):
        return 'worseCandidateAccepted'
    else:
        return 'worseCandidateRejected'
    
incumbent = []
incumbentObjective = 0 
currentSolution = []
currentSolutionObjective = 0 
def executeAnnealingAlgorithm():
    #initializeTemperatureSchedule
    temperatureSchedule =  initializeTemperatureSchedule(T, alpha, k)
    print('Temperature schedule: ', temperatureSchedule)
    
    #construct initial incumbent and initial solution
    incumbent = setIncumbent()
    currentSolution = incumbent
    
    #initialize incumbent and initial solution objective values
    incumbentObjective = checkObjective(incumbent)
    currentSolutionObjective = checkObjective(incumbent)
    
    print('Incubent: ',incumbent)
    print('Incubent Objective: ',incumbentObjective)
    print('Current Solution: ',currentSolution)
    print('Current Solution Objective: ',currentSolutionObjective, '\n')
    
    for temperature in temperatureSchedule:
        for index in range(m):
            #generate a candidate solution
            candidateSolution = generateFeasibleSolution()
            candidateSolutionObjective = checkObjective(candidateSolution)
            print('Candidate Solution: ',candidateSolution)
            print('Candidate Solution Objective: ', candidateSolutionObjective)
            print ('Temperature: ', temperature)
    
            #accept candidate solution to be current solution with some degree of probability
            acceptanceResult = acceptCandidate(currentSolutionObjective, candidateSolutionObjective, temperature)
            print(acceptanceResult, '\n')
            if (acceptanceResult == 'betterCandidateAccepted'):
                currentSolution = candidateSolution
                currentSolutionObjective = candidateSolutionObjective
                incumbent = candidateSolution
                incumbentObjective = candidateSolutionObjective
            elif (acceptanceResult == 'worseCandidateAccepted'):
                currentSolution = candidateSolution
                currentSolutionObjective = candidateSolutionObjective
            elif ('worseCandidateRejected'):
                print('no change')
    print('Final Incumbent Solution: ', incumbent)
    print('Final Incumbent Objective Solution: ', incumbentObjective)

executeAnnealingAlgorithm()
    
    

Temperature schedule:  [100, 20.0, 4.0, 0.8]
Incubent:  [1, 1, 0, 1]
Incubent Objective:  6
Current Solution:  [1, 1, 0, 1]
Current Solution Objective:  6 

Candidate Solution:  [1, 1, 0, 1]
Candidate Solution Objective:  6
Temperature:  100
betterCandidateAccepted 

Candidate Solution:  [1, 1, 1, 0]
Candidate Solution Objective:  104
Temperature:  100
betterCandidateAccepted 

Candidate Solution:  [0, 1, 1, 1]
Candidate Solution Objective:  104
Temperature:  20.0
betterCandidateAccepted 

Candidate Solution:  [1, 1, 0, 1]
Candidate Solution Objective:  6
Temperature:  20.0
worseCandidateRejected 

no change
Candidate Solution:  [0, 1, 1, 1]
Candidate Solution Objective:  104
Temperature:  4.0
betterCandidateAccepted 

Candidate Solution:  [1, 1, 0, 1]
Candidate Solution Objective:  6
Temperature:  4.0
worseCandidateRejected 

no change
Candidate Solution:  [1, 0, 1, 1]
Candidate Solution Objective:  104
Temperature:  0.8
betterCandidateAccepted 

Candidate Solution:  [0, 1, 1, 1]
Cand